In [2]:
# miscellaneous
import matplotlib.pyplot as plt
from glob import glob #library that helps us search for files
import scipy
import random
from datetime import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
from scipy.special import inv_boxcox
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

# preprocessing
from category_encoders import OneHotEncoder,TargetEncoder,OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, FunctionTransformer, PowerTransformer,LabelEncoder, MaxAbsScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score,cross_val_predict, KFold, StratifiedKFold
from sklearn.compose import ColumnTransformer

# models
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression,BayesianRidge, ElasticNet, Lasso
from sklearn.dummy import DummyClassifier
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import clone

# metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score
from sklearn.metrics import log_loss, confusion_matrix, ConfusionMatrixDisplay, classification_report,RocCurveDisplay
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.utils.validation import check_is_fitted

# feature selection / data sampling
from sklearn.feature_selection import RFE, SelectKBest, f_classif, SelectFromModel, VarianceThreshold
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.inspection import permutation_importance

# saving model
import pickle,joblib
import boto3

# data settings
pd.pandas.set_option('display.max_rows',None)
pd.pandas.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)


from sklearn import set_config
set_config(transform_output = 'pandas')

import warnings
warnings.simplefilter(action="ignore")


import json

# custom package
import importlib
import package
importlib.reload(package)

from package.data_retrieval.data_retrieval import s3_retrieval, write_to_s3
from package.eda.data_exploration import correlation

# Read Config

In [3]:
config_path = "C:/Users/Oamen/OneDrive/Documents/DATA PROJECTS/Insurance_claim_prediction_porto/config.json"

In [4]:
# Opening JSON file
with open(config_path) as file:
    
    config = json.load(file)

In [5]:
config['EDA']['config_path'] = config_path

# Read Data

In [6]:
# get access key
secret = pd.read_csv("C:/Users/Oamen/Downloads/jupyter_iam_accessKeys.csv")

In [7]:
# create S3 client object 
s3_client = boto3.client('s3',aws_access_key_id=secret['Access key ID'][0],aws_secret_access_key=secret['Secret access key'][0])

In [11]:
bucket_name = config['EDA']['bucket_name'] 

full_train_key = config['EDA']['full_train_path'] 
test_key = config['EDA']['test_path'] 
validation_key = config['EDA']['validation_path'] 
red_train_key = config['EDA']['red_train_path']

In [12]:
full_train = s3_retrieval(s3_client, bucket_name, full_train_key)

Retrieved object
Read bytes
Success!


In [13]:
test = s3_retrieval(s3_client, bucket_name, test_key)

Retrieved object
Read bytes
Success!


In [14]:
validation = s3_retrieval(s3_client, bucket_name, validation_key)

Retrieved object
Read bytes
Success!


In [15]:
reduced_train = s3_retrieval(s3_client, bucket_name, red_train_key)

Retrieved object
Read bytes
Success!


In [16]:
categorical_columns = config['EDA']['categorical_columns'] 

numerical_columns = config['EDA']['numerical_columns'] 

target = config['EDA']['target']

all_columns = categorical_columns + numerical_columns + [target]

In [17]:
# filter data

reduced_train = reduced_train[all_columns]
full_train = full_train[all_columns]
validation = validation[all_columns]
test = test[all_columns]

In [44]:
# set discrete to integer dtype

for column in categorical_columns:
    
    reduced_train[column] = reduced_train[column].apply(lambda x: x if pd.isnull(x) else str(x))
    full_train[column] = full_train[column].apply(lambda x: x if pd.isnull(x) else str(x))
    validation[column] = validation[column].apply(lambda x: x if pd.isnull(x) else str(x))
    test[column] = test[column].apply(lambda x: x if pd.isnull(x) else str(x))

In [45]:
reduced_train[:1]

,ps_ind_01,ps_ind_02_cat,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,ps_ind_11_bin,ps_ind_12_bin,ps_ind_13_bin,ps_ind_14,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_car_02_cat,ps_car_03_cat,ps_car_04_cat,ps_car_05_cat,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_09,ps_calc_12,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_ind_03,ps_ind_15,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_06_cat,ps_car_11_cat,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_08,ps_calc_10,ps_calc_11,ps_calc_13,ps_calc_14,target
0,3,2,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,-1,0,-1,1,1,2,1,3,0,0,0,3,4,8,5,3,0,0,1,1,0,1,0,2,7,0.5,1.040132,7,1,64,0.316228,0.610209,0.361939,2.828427,9,11,7,6,8,0


In [20]:
config['Model_Selection'] = {}
config['Model_Selection']['input_features'] = all_columns

# Dummy/Baseline Model

In [46]:
ytrain_dummy, xtrain_dummy = reduced_train[config['EDA']['target']], reduced_train.drop(config['EDA']['target'], axis = 1)

In [47]:
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('encoder',OneHotEncoder())
])

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'median'))
])

dummy_pipeline = ColumnTransformer([
    ('cat_pipeline', cat_pipeline, categorical_columns),
    ('num_pipeline', num_pipeline, numerical_columns),
    
], remainder = 'passthrough', verbose_feature_names_out = False)

In [48]:
xtrain_dummy = dummy_pipeline.fit_transform(xtrain_dummy)

In [49]:
xtrain_dummy[:1]

,ps_ind_01_1,ps_ind_01_2,ps_ind_01_3,ps_ind_01_4,ps_ind_01_5,ps_ind_01_6,ps_ind_01_7,ps_ind_01_8,ps_ind_02_cat_1,ps_ind_02_cat_2,ps_ind_02_cat_3,ps_ind_02_cat_4,ps_ind_02_cat_5,ps_ind_04_cat_1,ps_ind_04_cat_2,ps_ind_04_cat_3,ps_ind_05_cat_1,ps_ind_05_cat_2,ps_ind_05_cat_3,ps_ind_05_cat_4,ps_ind_05_cat_5,ps_ind_05_cat_6,ps_ind_05_cat_7,ps_ind_05_cat_8,ps_ind_06_bin_1,ps_ind_06_bin_2,ps_ind_07_bin_1,ps_ind_07_bin_2,ps_ind_08_bin_1,ps_ind_08_bin_2,ps_ind_09_bin_1,ps_ind_09_bin_2,ps_ind_10_bin_1,ps_ind_10_bin_2,ps_ind_11_bin_1,ps_ind_11_bin_2,ps_ind_12_bin_1,ps_ind_12_bin_2,ps_ind_13_bin_1,ps_ind_13_bin_2,ps_ind_14_1,ps_ind_14_2,ps_ind_14_3,ps_ind_14_4,ps_ind_14_5,ps_ind_16_bin_1,ps_ind_16_bin_2,ps_ind_17_bin_1,ps_ind_17_bin_2,ps_ind_18_bin_1,ps_ind_18_bin_2,ps_reg_01_1,ps_car_02_cat_1,ps_car_02_cat_2,ps_car_02_cat_3,ps_car_03_cat_1,ps_car_03_cat_2,ps_car_03_cat_3,ps_car_04_cat_1,ps_car_04_cat_2,ps_car_04_cat_3,ps_car_04_cat_4,ps_car_04_cat_5,ps_car_04_cat_6,ps_car_04_cat_7,ps_car_04_cat_8,ps_car_04_cat_9,ps_car_04_cat_10,ps_car_05_cat_1,ps_car_05_cat_2,ps_car_05_cat_3,ps_car_07_cat_1,ps_car_07_cat_2,ps_car_07_cat_3,ps_car_08_cat_1,ps_car_08_cat_2,ps_car_09_cat_1,ps_car_09_cat_2,ps_car_09_cat_3,ps_car_09_cat_4,ps_car_09_cat_5,ps_car_09_cat_6,ps_car_10_cat_1,ps_car_10_cat_2,ps_car_10_cat_3,ps_car_11_1,ps_car_11_2,ps_car_11_3,ps_car_11_4,ps_car_11_5,ps_calc_01_1,ps_calc_02_1,ps_calc_03_1,ps_calc_04_1,ps_calc_04_2,ps_calc_04_3,ps_calc_04_4,ps_calc_04_5,ps_calc_04_6,ps_calc_05_1,ps_calc_05_2,ps_calc_05_3,ps_calc_05_4,ps_calc_05_5,ps_calc_05_6,ps_calc_05_7,ps_calc_06_1,ps_calc_06_2,ps_calc_06_3,ps_calc_06_4,ps_calc_06_5,ps_calc_06_6,ps_calc_06_7,ps_calc_06_8,ps_calc_06_9,ps_calc_06_10,ps_calc_06_11,ps_calc_07_1,ps_calc_07_2,ps_calc_07_3,ps_calc_07_4,ps_calc_07_5,ps_calc_07_6,ps_calc_07_7,ps_calc_07_8,ps_calc_07_9,ps_calc_07_10,ps_calc_09_1,ps_calc_09_2,ps_calc_09_3,ps_calc_09_4,ps_calc_09_5,ps_calc_09_6,ps_calc_09_7,ps_calc_09_8,ps_calc_12_1,ps_calc_12_2,ps_calc_12_3,ps_calc_12_4,ps_calc_12_5,ps_calc_12_6,ps_calc_12_7,ps_calc_12_8,ps_calc_12_9,ps_calc_12_10,ps_calc_12_11,ps_calc_15_bin_1,ps_calc_15_bin_2,ps_calc_16_bin_1,ps_calc_16_bin_2,ps_calc_17_bin_1,ps_calc_17_bin_2,ps_calc_18_bin_1,ps_calc_18_bin_2,ps_calc_19_bin_1,ps_calc_19_bin_2,ps_calc_20_bin_1,ps_calc_20_bin_2,ps_ind_03,ps_ind_15,ps_reg_02,ps_reg_03,ps_car_01_cat,ps_car_06_cat,ps_car_11_cat,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_08,ps_calc_10,ps_calc_11,ps_calc_13,ps_calc_14
0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0,2.0,7.0,0.5,1.040132,7.0,1.0,64.0,0.316228,0.610209,0.361939,2.828427,9.0,11.0,7.0,6.0,8.0


In [61]:
dummy = DummyClassifier(strategy = 'stratified', random_state = 0)

dummy.fit(xtrain_dummy,ytrain_dummy)

DummyClassifier(random_state=0, strategy='stratified')

In [62]:
dummy_pred = dummy.predict(xtrain_dummy)

In [63]:
metrics_dummy = pd.DataFrame({
    'Model': 'Baseline',
    'AUC': roc_auc_score(ytrain_dummy,dummy_pred),
    'F1': f1_score(ytrain_dummy,dummy_pred),
    'Accuracy': accuracy_score(ytrain_dummy,dummy_pred),
    'Precision': precision_score(ytrain_dummy,dummy_pred),
    'Recall': recall_score(ytrain_dummy,dummy_pred),    
}, index = [0])

metrics_dummy

,Model,AUC,F1,Accuracy,Precision,Recall
0,Baseline,0.500496,0.037427,0.929788,0.037403,0.037451


# Experiment 0